In [1]:
# Meta code to wrap into slides
# `panel serve app.ipynb --autoreload`

import panel as pn

pn.extension(
    template='slides', css_files=[
        'https://fonts.googleapis.com/css?family=Inter'
    ], raw_css=[
        'html, body { font-family: "Inter",sans-serif; }',
        ':root {--r-background-color: rgb(16, 39, 47) }',
        "h1 { text-align: left; font-size: 3em; }",
        "h2 { text-align: left; font-size: 3em; }",
        "li { text-align: left; font-size: 2em; }",
        "pre { text-align: left; }",
    ],
    theme='dark'
)

pn.state.template.param.update(
    design=pn.theme.Material,
    header_background='white',
    logo='anaconda.png',
    title='Improve ML performance with hvplot',
)

pn.state.template.config.param.update(
    raw_css=[
        "#header { height: 0; padding: 20px; }",
        "li { text-align: left; }",
        "p { margin-block-start: 0.5em; margin-block-end: 0.2em}",
    ],
    css_files=[pn.io.resources.CSS_URLS['font-awesome']]
)

slide = lambda *objs: pn.Column(
    pn.Column(*objs), sizing_mode='stretch_both', min_height=600, styles={
        'display': 'flex', 'height': '100%', 'align-items': 'center'
    }
)

def header(text, size='4em', **kwargs):
    return HTML(f'<span>{text}</span>', styles={'font-size': size, 'font-weight': 'bold'}, **kwargs)

def text_fragment(text, size='0.5em', **kwargs):
    return Markdown(
        text,
        styles={'font-size': size, 'font-weight': 'bold'},
        tags=['fragment'], **kwargs
    )


def ends(df):
    try:
        return pn.Column(df.iloc[:5, :5], "...", df.iloc[-5:, -5:])
    except:
        return pn.Column(df.iloc[:5], "...", df.iloc[-5:])

In [2]:
slide(
    pn.Column(
        """
        # Improving ML performance with hvplot

        - hvplot makes visualizing your data easy; simply set kwargs
        - find the most influential variables for feature selection
        - help improve model performance
        """,
        pn.Row(
            pn.panel("heatmap.png", width=300),
            pn.panel("katrina.png", width=300),
            pn.panel("tracks_climatology.png", width=300)
        ),
    ).servable()
)


BokehModel(combine_events=True, render_bundle={'docs_json': {'a02188ed-b15c-4d3f-8078-325c655f374f': {'version…

In [3]:
import os
os.system("wget -nc https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/csv/ibtracs.since1980.list.v04r00.csv")

slide(
    pn.Column(
        """
        ## Download the data

        - Archive of tropical cyclone records (TC) since 1980
        - Tracks, intensity, location, wind speed, central pressure, etc
        ```python
        wget -nc https://www.ncei.noaa.gov/data/international-best-track-archive-for-
        climate-stewardship-ibtracs/v04r00/access/csv/ibtracs.since1980.list.v04r00.csv
        ```
        """
    ).servable()
)

File ‘ibtracs.since1980.list.v04r00.csv’ already there; not retrieving.



BokehModel(combine_events=True, render_bundle={'docs_json': {'bcb8017e-c7a5-4dac-8f87-a75e218f02dd': {'version…

In [4]:
import pandas as pd

df = pd.read_csv("ibtracs.since1980.list.v04r00.csv")

slide(
    pn.Column(
        pn.Row(
            """
            ## Read / show the data

            - Ensure data as expected
            - Use display for prettier tables
            - All are `objects` (strings)
            """,
            """
            ```python
            import pandas as pd

            pd.set_option('display.max_columns', None)
            pd.set_option('display.max_rows', None)

            df = pd.read_csv("ibtracs.since1980.list.v04r00.csv")
            display(df.dtypes[:8], df.head())
            ```
            """,
        ),
        pn.Row(ends(df.dtypes), ends(df)),
    ).servable()
)


/var/folders/8h/mk8mz7qn3_325ztm7pgb4ld00000gp/T/ipykernel_84938/4267609727.py:3: DtypeWarning: Columns (1,2,8,9,14,19,20,161,162) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ibtracs.since1980.list.v04r00.csv")


BokehModel(combine_events=True, render_bundle={'docs_json': {'13d2a0a0-3593-4d27-b201-3e2caa2c9dfe': {'version…

In [5]:
df = pd.read_csv("ibtracs.since1980.list.v04r00.csv", parse_dates=["ISO_TIME"], skiprows=[1])

icols = [19, 20, 161, 162]
for icol in icols:
    col = df.iloc[:, icol].name
    df[col] = pd.to_numeric(df[col], errors="coerce")

slide(
    pn.Column(
        """
        ## Clean the data

        - Remove second header row
        - Fix data types (object to numeric)

        ```python
        df = pd.read_csv("ibtracs.since1980.list.v04r00.csv", parse_dates=["ISO_TIME"], skiprows=[1])
    
        icols = [19, 20, 161, 162]
        for icol in icols:
            col = df.iloc[:, icol].name
            df[col] = pd.to_numeric(df[col], errors="coerce")
    
        display(df.dtypes[:8], df.head())
        ```
        """,
        pn.Row(ends(df.dtypes), ends(df)),
    ).servable()
)


/var/folders/8h/mk8mz7qn3_325ztm7pgb4ld00000gp/T/ipykernel_84938/1233139657.py:1: DtypeWarning: Columns (19,20,161,162) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ibtracs.since1980.list.v04r00.csv", parse_dates=["ISO_TIME"], skiprows=[1])


BokehModel(combine_events=True, render_bundle={'docs_json': {'fec0332c-86a2-4d8b-b31d-49b830034de4': {'version…

In [6]:
df.to_parquet("ibtracs.parquet")

slide(
    pn.Column(
        """
        ## Convert the format

        - Minimal effort for improved efficiency

        ```python
        df.to_parquet("ibtracs.parquet")
        ```
        """,
    ).servable()
)


BokehModel(combine_events=True, render_bundle={'docs_json': {'5e3c9e88-e8fa-434e-b3c6-feea246cb6a1': {'version…

In [7]:
import duckdb
import hvplot.pandas

katrina_df = duckdb.execute(
    """
    SELECT LON, LAT, USA_SSHS, USA_WIND, USA_PRES, ISO_TIME
    FROM 'ibtracs.parquet'
    WHERE NAME == 'KATRINA' AND SEASON == 2005
    """
).fetchdf()

katrina_points = katrina_df.hvplot.points(
    x="LON",
    y="LAT",
    color="USA_SSHS",
    hover_cols=["USA_WIND", "USA_PRES", "ISO_TIME"],
    tiles=True,
    aspect=1
).opts(responsive=True)

slide(
    pn.Row(
        '''
        ## Plot Katrina track

        - Using duckdb + SQL to read parquet
        - Easier to type and read and more performant
        - `fetchdf()` serializes into `pd.DataFrame`
        - Effortlessly plot by setting keywords

        ```python
        import duckdb
        import hvplot.pandas

        katrina_df = duckdb.execute(
            """
            SELECT LON, LAT, USA_SSHS, USA_WIND, USA_PRES, ISO_TIME
            FROM 'ibtracs.parquet'
            WHERE NAME == 'KATRINA' AND SEASON == 2005
            """
        ).fetchdf()

        # equivalent to the above
        # df = pd.read_parquet("ibtracs.parquet")
        # katrina_df = df.loc[
        #   (df["NAME"] == "KATRINA") &
        #   (df["SEASON"] == 2005),
        #   ["LON", "LAT", "USA_SSHS", "USA_WIND", "USA_PRES", "ISO_TIME"]
        # ]

        katrina_df.hvplot.points(
            x="LON",
            y="LAT",
            color="USA_SSHS",
            hover_cols=["USA_WIND", "USA_PRES", "ISO_TIME"],
            tiles=True,
        )
        ```
        ''',
    ).servable()
)


slide(
    pn.pane.HoloViews(katrina_points).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'17028035-153f-4296-a360-9f9193c689b7': {'version…

In [8]:
import duckdb
import hvplot.pandas
import holoviews as hv

df = duckdb.query(
    """
    SELECT LON, LAT FROM 'ibtracs.parquet'
    """
).fetchdf()

climatology_points = df.hvplot.points(
    x="LON",
    y="LAT",
    cmap="viridis",
    rasterize=True,
    tiles=True,
    x_sampling=1,
    y_sampling=1,
    aspect=1,
).opts(xlim=(-179, 179), responsive=True)

slide(
    pn.Column(
        '''
        ## Plot tracks climatology

        - `rasterize` can visualize billions of points
        - Prevents overplotting--converts into 2D histogram

        ```python
        import duckdb
        import hvplot.pandas

        df = duckdb.query(
            """
            SELECT LON, LAT FROM 'ibtracs.parquet'
            """
        ).fetchdf()

        df.hvplot.points(
            x="LON",
            y="LAT",
            cmap="viridis",
            rasterize=True,
            coastline=True,
            global_extent=True,
        )
        ```
        ''',
    ).servable()
)

slide(
    pn.pane.HoloViews(climatology_points).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'7290430d-c20c-4e6a-a61b-0d70d13c1bed': {'version…

In [9]:
slide(
    pn.Row(
        '''
        ## Initial motivation

        "El Niño events generally suppress Atlantic hurricane<br>activity
        with fewer hurricanes than normal in the Atlantic basin<br>
        during the peak of Atlantic hurricane season"
        [[Source](https://www.weather.gov/jan/el_nino_and_la_nina)].
        ''',
        pn.panel("https://www.weather.gov/images/jan/ElNino_LaNina/elninowxpatterns.gif"),
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e890bf9d-6d08-494b-a0f6-7c683b94e0c8': {'version…

In [10]:
atlantic_df = duckdb.execute("""
    SELECT *
    FROM ibtracs.parquet
    WHERE
        NAME != 'NOT_NAMED' AND
        USA_ATCF_ID LIKE 'AL%' AND
        SEASON < 2023 AND
        DATE_PART('month', ISO_TIME) >= 6 AND
        DATE_PART('month', ISO_TIME) <= 11
""").fetchdf()

atlantic_points = atlantic_df.hvplot.points(
    x="LON",
    y="LAT",
    cmap="viridis",
    rasterize=True,
    tiles=True,
    x_sampling=1,
    y_sampling=1,
    aspect=1,
).opts(xlim=(-179, 179), responsive=True)


slide(
    pn.Column(
        '''
        ## Select Atlantic hurricane season

        - The six-month season runs from June 1 to November 30 [[Source](https://www.noaa.gov/news-release/2023-atlantic-hurricane-season-outlook)].
        - Subset named hurricanes in the Atlantic basin during season only
        - Verify queried as expected by visualizing

        ```python
        atlantic_df = duckdb.execute("""
            SELECT *
            FROM ibtracs.parquet
            WHERE
                NAME != 'NOT_NAMED' AND
                USA_ATCF_ID LIKE 'AL%' AND
                SEASON < 2023 AND
                DATE_PART('month', ISO_TIME) >= 6 AND
                DATE_PART('month', ISO_TIME) <= 11
        """).fetchdf()

        # equivalent to the above
        # df = pd.read_parquet("ibtracs.parquet")
        # atlantic_df = df.loc[
        #   (df["USA_SSHS"] > 0) &
        #   (df["USA_ATCF_ID"].str.startswith("AL"))
        # ]

        atlantic_df.hvplot.points(
            x="LON",
            y="LAT",
            cmap="viridis",
            rasterize=True,
            coastline=True,
            x_sampling=1,
            y_sampling=1,
        )
        ```
        ''',
    ).servable()
)

slide(
    pn.pane.HoloViews(atlantic_points).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'aeb994c4-f543-40bc-bd23-914deeca11f7': {'version…

In [11]:
atlantic_count_df = duckdb.execute("""
    SELECT SEASON, COUNT(DISTINCT NAME) AS unique_names
    FROM ibtracs.parquet
    WHERE
        NAME != 'NOT_NAMED' AND
        USA_ATCF_ID LIKE 'AL%' AND
        SEASON < 2023 AND
        DATE_PART('month', ISO_TIME) >= 6 AND
        DATE_PART('month', ISO_TIME) <= 11
    GROUP BY SEASON;
""").fetchdf()

count_plot = atlantic_count_df.hvplot("SEASON", "unique_names")

slide(
    pn.Column(
        '''
        ## Calculate Atlantic hurricane season count

        - Names are unique per season (increments by alphabet)

        ```python
        atlantic_count_df = duckdb.execute("""
            SELECT SEASON, COUNT(DISTINCT NAME) AS unique_names
            FROM ibtracs.parquet
            WHERE
                NAME != 'NOT_NAMED' AND
                USA_ATCF_ID LIKE 'AL%' AND
                SEASON < 2023 AND
                DATE_PART('month', ISO_TIME) >= 6 AND
                DATE_PART('month', ISO_TIME) <= 11
            GROUP BY SEASON;
        """).fetchdf()

        atlantic_count_df.hvplot("SEASON", "unique_names")
        ''',
        pn.pane.HoloViews(count_plot)
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'8d5e6113-bb3c-4597-af17-707286dac84b': {'version…

In [12]:
nino_df = pd.read_csv('https://raw.githubusercontent.com/ahuang11/oni/master/nino_ml.csv', index_col=0, parse_dates=True)
nino_df = nino_df.rename_axis("date").dropna()

slide(
    pn.Column(
        '''
        ## Get data relevant to Nino

        t300 - depth averaged temps up from 0 to 300m
        wwv - warm water volume
        u850 - 850 mb trade wind index
        *_e - east
        *_w - west
        *_c - central
        *_anom - anomaly
        *_norm - standardized

        ```python
        nino_df = pd.read_csv('https://raw.githubusercontent.com/ahuang11/oni/master/nino_ml.csv', index_col=0, parse_dates=True)
        nino_df = nino_df.rename_axis("date").dropna()

        display(nino_df.head())
        ''',
        ends(nino_df),
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'612d0cb6-3c21-48ee-96cf-c4e499a91326': {'version…

In [13]:
nino_df.to_parquet("nino.parquet")

nino_spring_df = duckdb.execute(
    """
    SELECT *, EXTRACT(MONTH FROM date) AS month, EXTRACT(YEAR FROM date) AS year
    FROM nino.parquet
    WHERE EXTRACT(MONTH FROM date) BETWEEN 1 AND 5;
    """
).fetchdf()

slide(
    pn.Row(
        '''
        ## Subset predictors
        
        - Select January to May (prior to peak season)

        ```python
        nino_df.to_parquet("nino.parquet")

        nino_spring_df = duckdb.execute(
            """
            SELECT *, EXTRACT(MONTH FROM date) AS month, EXTRACT(YEAR FROM date) AS year
            FROM nino.parquet
            WHERE EXTRACT(MONTH FROM date) BETWEEN 1 AND 5;
            """
        ).fetchdf()

        display(nino_spring_df.head())
        ''',
        pn.Column(
            pn.panel("diagram.png", width=500),
            ends(nino_spring_df),
        )
    ).servable()
)

Task exception was never retrieved
future: <Task finished name='Task-12' coro=<Callback.process_on_change() done, defined at /Users/ahuang/miniconda3/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py:322> exception=UnsetValueError("figure(id='f2b489ba-f955-41c5-a7c0-5b021f8ebb52', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/Users/ahuang/miniconda3/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 340, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/Users/ahuang/miniconda3/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 248, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/Users/ahuang/miniconda3/lib/python3.10/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descriptors.UnsetValueError: figure(id='f2b489

BokehModel(combine_events=True, render_bundle={'docs_json': {'7d91270c-d869-400d-b449-849858c81afa': {'version…

In [14]:
ml_df = (
    atlantic_count_df.set_index("SEASON").join(nino_spring_df.set_index("year"))
).reset_index(names=["year"])

slide(
    pn.Column(
        '''
        ## Combine the two dataframes

        - Use year as the primary key

        ```python
        ml_df = (
            atlantic_count_df.set_index("SEASON").join(nino_spring_df.set_index("year"))
        ).reset_index(names=["year"])
        
        display(ml_df.head())
        ```
        ''',
        ends(ml_df),
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'46e46cc8-130c-4ce3-b380-464098177b2c': {'version…

In [15]:
corr_df = (
    ml_df.groupby("month")
    .corr(numeric_only=True)["unique_names"]
    .sort_values()
    .rename_axis(["month", "parameter"])
)

corr_heatmap = corr_df.hvplot.heatmap(
    "month",
    "parameter",
    "unique_names",
    cmap="RdBu_r",
    colorbar=True,
    symmetric=True,
    height=800,
).opts(color_levels=12)

slide(
    pn.Row(
        '''
        # Explore features

        - Feature selection helps performance
        - Select the ones with highest correlation
        - Nino indices are not extremely correlated
        - Zonal winds at 850mb are quite correlated

        ```python
        corr_df = (
            ml_df.groupby("month")
            .corr(numeric_only=True)["unique_names"]
            .sort_values()
            .rename_axis(["month", "parameter"])
        )
        corr_df.hvplot.heatmap(
            "month",
            "parameter",
            "unique_names",
            cmap="RdBu_r",
            colorbar=True,
            symmetric=True,
            height=800,
        ).opts(color_levels=12)
        ```
        ''',
        pn.panel(corr_heatmap)
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'81439d4a-09e1-4b2e-8382-efd1534c670e': {'version…

In [16]:
ml_month_dfs = []

for month in range(4, 6):
    ml_month_df = ml_df.loc[ml_df["month"] == month].drop(columns=["month"])
    ml_month_df = ml_month_df.set_index(["year", "unique_names"])
    ml_month_df.columns = ml_month_df.columns + f"_m{month}"
    ml_month_dfs.append(ml_month_df)

ml_month_df = pd.concat(ml_month_dfs, axis=1).reset_index()

slide(
    pn.Column(
        '''
        ## Re-orient the data

        - Convert months into separate predictor columns

        ```python
        ml_month_dfs = []
        for month in range(4, 6):
            ml_month_df = ml_df.loc[ml_df["month"] == month].drop(columns=["month"])
            ml_month_df = ml_month_df.set_index(["year", "unique_names"])
            ml_month_df.columns = ml_month_df.columns + f"_m{month}"
            ml_month_dfs.append(ml_month_df)

        ml_month_df = pd.concat(ml_month_dfs, axis=1).reset_index()
        ```
        ''',
        ends(ml_month_df),
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'bdcef2f3-264e-4f74-af0c-a1310db30dfc': {'version…

In [17]:
ml_param_df = ml_month_df.filter(
    regex="t300_w|u850_c|olr|t300_e|wwv_e|year|month|unique_names"
)

slide(
    pn.Column(
        '''
        ## Subset the best features

        - Rather than manually typing each specific column name, filter by regex
        ```python
        ml_param_df = ml_month_df.filter(
            regex="t300_w|u850_c|olr|t300_e|wwv_e|year|month|unique_names"
        )

        display(ml_param_df.head())
        ```
        ''',
        ends(ml_param_df),
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'8fba02c9-88c0-487b-8374-52ef424f75a2': {'version…

In [18]:
import statsmodels.api as sm

def run_model(model, X, y, train_index, val_index, **model_kwargs):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    X_train = sm.add_constant(X_train)
    model = model(y_train, X_train, **model_kwargs)
    fitted_model = model.fit()

    X_val = sm.add_constant(X_val)
    y_pred_val = fitted_model.predict(X_val)

    val_df = pd.DataFrame(
        {"year": X_val["year"], "actual": y_val, "prediction": y_pred_val}
    ).sort_values("year")
    return val_df

def score_output(val_df):
    corr = val_df["actual"].corr(val_df["prediction"])
    rmse = ((val_df["actual"] - val_df["prediction"]) ** 2).mean() ** 0.5
    return pd.Series([corr, rmse], index=["corr", "rmse"])

slide(
    pn.Row(
        '''
        ## Create reusable ML functions

        - Splits dataset into training and validation
        - Fits the model
        - Makes prediction
        - Stores output for scoring
        ''',
        '''
        ```python
        import statsmodels.api as sm

        def run_model(model, X, y, train_index, val_index, **model_kwargs):
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]

            X_train = sm.add_constant(X_train)
            model = model(y_train, X_train, **model_kwargs)
            fitted_model = model.fit()

            X_val = sm.add_constant(X_val)
            y_pred_val = fitted_model.predict(X_val)

            val_df = pd.DataFrame(
                {"year": X_val["year"], "actual": y_val, "prediction": y_pred_val}
            ).sort_values("year")
            return val_df


        def score_output(val_df):
            corr = val_df["actual"].corr(val_df["prediction"])
            rmse = ((val_df["actual"] - val_df["prediction"]) ** 2).mean() ** 0.5
            return pd.Series([corr, rmse], index=["corr", "rmse"])
        ```
        ''',
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'a6331c80-e20c-472a-8797-13afb9c2aa51': {'version…

In [19]:
from sklearn.model_selection import KFold

X = ml_param_df.drop(columns=["unique_names"])
y = ml_param_df["unique_names"]

num_folds = 4  # Adjust the number of folds as needed
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
val_dfs = []
for i, (train_index, val_index) in enumerate(kf.split(X)):
    val_df = run_model(sm.OLS, X, y, train_index, val_index)
    val_df["fold"] = i
    val_dfs.append(val_df)

val_df = pd.concat(val_dfs).sort_values("year")

score_df = (val_df.groupby("fold").apply(score_output))
val_plot = val_df.hvplot("year", ["actual", "prediction"], title="With Feature Selection")


slide(
    pn.Column(
        '''
        # Run linear regression and score

        - Split dataset into X (predictors) and Y (goal)
        - Run K-Fold for reliability and preventing overfitting

        ```python
        from sklearn.model_selection import KFold

        X = ml_param_df.drop(columns=["unique_names"])
        y = ml_param_df["unique_names"]

        num_folds = 4  # Adjust the number of folds as needed
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
        val_dfs = []
        for i, (train_index, val_index) in enumerate(kf.split(X)):
            val_df = run_model(sm.OLS, X, y, train_index, val_index)
            val_df["fold"] = i
            val_dfs.append(val_df)

        val_df = pd.concat(val_dfs).sort_values("year")

        score_df = (val_df.groupby("fold").apply(score_output))
        val_plot = val_df.hvplot("year", ["actual", "prediction"])
        display(val_plot, score_df)
        ```
        ''',
    ).servable()
)

slide(
    pn.Row(
        val_plot, score_df,
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'d8ead61c-a1a6-44d7-b46d-bc95e8466706': {'version…

In [20]:
X = ml_month_df.drop(columns=["unique_names"]).select_dtypes("number")
y = ml_month_df["unique_names"]

num_folds = 4  # Adjust the number of folds as needed
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
val_dfs = []
for i, (train_index, val_index) in enumerate(kf.split(X)):
    val_df = run_model(sm.OLS, X, y, train_index, val_index)
    val_df["fold"] = i
    val_dfs.append(val_df)

val_df = pd.concat(val_dfs).sort_values("year")

new_score_df = (val_df.groupby("fold").apply(score_output))
new_val_plot = val_df.hvplot("year", ["actual", "prediction"], title="No Feature Selection")


slide(
    pn.Column(
        '''
        ## Re-run and score w/o feature selection

        - Other features are noise
        - This lowers the skill score

        ```python
        X = ml_month_df.drop(columns=["unique_names"]).select_dtypes("number")
        y = ml_month_df["unique_names"]

        num_folds = 4  # Adjust the number of folds as needed
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
        val_dfs = []
        for i, (train_index, val_index) in enumerate(kf.split(X)):
            val_df = run_model(sm.OLS, X, y, train_index, val_index)
            val_df["fold"] = i
            val_dfs.append(val_df)

        val_df = pd.concat(val_dfs).sort_values("year")

        score_df = (val_df.groupby("fold").apply(score_output))
        val_plot = val_df.hvplot("year", ["actual", "prediction"])
        display(val_plot, score_df)
        ```
        ''',
    ).servable()
)

slide(
    pn.Column(
        pn.Row(
            val_plot, score_df,
        ),
        pn.Row(
            new_val_plot, new_score_df,
        )
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'8b8c0d98-8f66-487e-8a72-c4510ab62703': {'version…

In [21]:
slide(
    pn.Row(
        '''
        ## Ideas to try

        - Different models (deep learning?)
        - Using other predictors (other oscillations)
        - Feature engineering (previous year's count)
        ''',
    ).servable()
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'c1eb69f1-13e9-4e56-82c8-68b32bc2c288': {'version…